In [122]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM coord"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=column_names)
cursor.close()
mydb.close()


In [ ]:
# ----------------------------------------------------OSRM Match service API Code ----------------------------------------------------#

import requests
import json
import mysql.connector
import numpy as np

def create_pairs(data):
    pairs = []
    first_lat_lng = f"{data.iloc[0]['longitude']},{data.iloc[0]['latitude']}"
    pairs.append(first_lat_lng)
    for i in range(4, len(data), 5):  # Starting from index 4, select every 5th latitude and longitude
        lat_lng = f"{data.iloc[i]['longitude']},{data.iloc[i]['latitude']}"
        pairs.append(lat_lng)
    last_lat_lng = f"{data.iloc[-1]['longitude']},{data.iloc[-1]['latitude']}"
    pairs.append(last_lat_lng)
    return pairs

def cpmini(data):
    pairs = []
    first_lat_lng = f"{data.iloc[0]['longitude']},{data.iloc[0]['latitude']}"
    
    pairs.append(first_lat_lng)
    for i in range(0, len(data)):  # Starting from index 0
        lat_lng = f"{data.iloc[i]['longitude']},{data.iloc[i]['latitude']}"
        pairs.append(lat_lng)
    last_lat_lng = f"{data.iloc[-1]['longitude']},{data.iloc[-1]['latitude']}"
    # if last_lat_lng not in pairs:
    pairs.append(last_lat_lng)
    return pairs

def send_osrm_request(pairs, trip_id, speed, recorded):
    radiuses = ['49'] * len(pairs)
    url = f"http://localhost:5000/match/v1/bicycle/{';'.join(pairs)}?overview=full&radiuses={';'.join(radiuses)}&geometries=geojson&annotations=true"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json() 
    
        node_id = ""
    
        for match in result['matchings']:
            for leg in match['legs']:
                for n_id in leg['annotation']['nodes']:
                    if node_id:
                        node_id += ","
                    node_id += str(n_id)
            # for point in match['geometry']['coordinates']:
        matched_coordinates.append({'trip_id': trip_id, 'response': result, 'speed': speed, 'recorded': recorded, 'node_id':node_id})
    else:
        print("Error:", response.status_code)
        result = response.json()
        print(result)
        print("Error in Trip_id", trip_id)
        error_trip_ids.append(trip_id)
        print(url)
        print(error_trip_ids)

    return matched_coordinates

matched_coordinates = []
error_trip_ids = []


# batch_data = df[df['trip_id'] == 2]

# pairs = create_pairs(batch_data)
# matched_coordinates_response = send_osrm_request(pairs)

# tid = df['trip_id'].unique()[1002:1003]
tid = ids_only_in_df1 
for i in tid:
    # batch_data = df[df['trip_id'] == i]
    batch_data = df[df['trip_id'] == i]

    if len(batch_data) == 1:

        if len(batch_data) == 1:
            rec = str(batch_data.iloc[0]['recorded'])  # Get the recorded time
            spd = str(batch_data.iloc[0]['speed'])
            node_id = "NA"
            resp = f"{batch_data.iloc[0]['latitude']},{batch_data.iloc[0]['longitude']}"
            matched_coordinates.append({'trip_id': trip_id, 'response': resp, 'speed': spd, 'recorded': rec, 'node_id':node_id})
            print("Response from OSRM API:", matched_coordinates)

    else:

        if len(batch_data) <= 9:
                splitted_dfs = batch_data
                print(f"DataFrames with less than 9 value", splitted_dfs)
                tstmp = ""
                sr = ""
                for k in range(0, len(splitted_dfs)):

                        recorded_time = str(splitted_dfs.iloc[k]['recorded'])  # Get the recorded time
                        if tstmp:  # Check if the string is not empty
                            tstmp += ","  # Add comma separator if needed
                        tstmp += recorded_time

                        speedrng = str(splitted_dfs.iloc[k]['speed'])  # Get the recorded time
                        if sr:  # Check if the string is not empty
                            sr += ","  # Add comma separator if needed
                        sr += speedrng
                
                trip_id = f"{splitted_dfs.iloc[0]['trip_id']}"
                pairs = cpmini(splitted_dfs)
                matched_coordinates = send_osrm_request(pairs, trip_id=trip_id, speed=sr, recorded=tstmp)
        else:

            if 100 >= len(batch_data) > 9:
                sdfs = np.array_split(batch_data, 1)

            elif 500 >= len(batch_data) > 100:
                sdfs = np.array_split(batch_data, 2)

            elif 1000 >= len(batch_data) > 500:
                sdfs = np.array_split(batch_data, 3)

            elif 5000 >= len(batch_data) > 1000:
                sdfs = np.array_split(batch_data, 15)

            elif 10000 >= len(batch_data) > 5000:
                sdfs = np.array_split(batch_data, 30)

            elif 15000 >= len(batch_data) > 10000:
                sdfs = np.array_split(batch_data, 40)

            else:
                sdfs = np.array_split(batch_data, 70)

            for j, df_part in enumerate(sdfs):
                
                print(f"DataFrame {j+1}:\n{df_part}\n")
                timestmp = ""
                speedrange = ""
                for k in range(0, len(df_part), 5):

                        recorded_time = str(df_part.iloc[k]['recorded'])  # Get the recorded time
                        if timestmp:  # Check if the string is not empty
                            timestmp += ","  # Add comma separator if needed
                        timestmp += recorded_time

                        speedrng = str(df_part.iloc[k]['speed'])  # Get the recorded time
                        if speedrange:  # Check if the string is not empty
                            speedrange += ","  # Add comma separator if needed
                        speedrange += speedrng
                
                trip_id = f"{df_part.iloc[0]['trip_id']}"
                # pairs = cp(df_part)
                # matched_coordinates = sor(pairs, trip_id=trip_id, speed=speedrange, recorded=timestmp)
                pairs = create_pairs(df_part)
                matched_coordinates = send_osrm_request(pairs, trip_id=trip_id, speed=speedrange, recorded=timestmp)
                # print("Response from OSRM API:", matched_coordinates_response)
                # print("Response from OSRM API:", matched_coordinates)

print("Response from OSRM API:", matched_coordinates)


In [ ]:
# -----------------------------Creating table--------------------------------#

import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

create_table_query = """
CREATE TABLE IF NOT EXISTS osr (
    trip_id INT,
    response LONGTEXT,
    speed TEXT,
    recorded TEXT,
    node_id TEXT
)
"""

# Execute the table creation query
cursor = mydb.cursor()
cursor.execute(create_table_query)

insert_query = "INSERT INTO osr (trip_id, response, speed, recorded, node_id) VALUES (%s, %s, %s, %s, %s)"

# Iterate over matched_coordinates and insert each record into the table
for record in matched_coordinates:
    response = json.dumps(record['response'])  # Convert response to JSON string
    cursor.execute(insert_query, (record['trip_id'], response, record['speed'], record['recorded'], record['node_id']))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()


In [ ]:


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Viewing Table # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM osr"
# sql_query = "SELECT * FROM osr WHERE trip_id BETWEEN 1299 AND 1302"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df_osr = pd.DataFrame(rows, columns=column_names)

# batch_data = df[df['trip_id'] == 1]
# print(len(batch_data['latitude']))

print(df_osr_small)
cursor.close()
mydb.close()

In [ ]:
# Print Unique Trip_ids
unique_count = df['trip_id'].nunique()
print("Number of unique values:", unique_count)

# Print the rows from coord table
batch_data = df[df['trip_id'] == 203]
print(batch_data)


# Print the Dcitionary of OSRM response stored
for record in matched_coordinates:
    trip_id = record['trip_id']
    response = json.dumps(record['response'])  # Convert response to JSON string
    speed = record['speed']
    recorded = record['recorded']
    node_id = record['node_id']
    print(trip_id, response, speed, recorded, node_id)

In [105]:
# -----------------------------updating table--------------------------------


import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

# Execute the table creation query
cursor = mydb.cursor()


insert_query = "INSERT INTO osr (trip_id, response, speed, recorded, node_id) VALUES (%s, %s, %s, %s, %s)"

# Iterate over matched_coordinates and insert each record into the table
for record in matched_coordinates:
    response = json.dumps(record['response'])  # Convert response to JSON string
    cursor.execute(insert_query, (record['trip_id'], response, record['speed'], record['recorded'], record['node_id']))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()
